In [1]:
# from decimal import Decimal
import polars as pl
from nautilus_trader.backtest.node import BacktestDataConfig
from nautilus_trader.backtest.node import BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.node import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig

# from nautilus_trader.model.data import Bar
# from nautilus_trader.model.data import BarType
from nautilus_trader.persistence.catalog import ParquetDataCatalog

## 2. Set up a Parquet data catalog

If everything worked correctly, you should be able to see a single EUR/USD instrument in the catalog.

In [2]:
# You can also use a relative path such as `ParquetDataCatalog("./catalog")`,
# for example if you're running this notebook after the data setup from the docs.
# catalog = ParquetDataCatalog.from_env()
catalog = ParquetDataCatalog("../data/binance/catalog")
catalog.instruments()

[CryptoPerpetual(id=ADAUSDT-PERP.BINANCE, raw_symbol=ADAUSDT, asset_class=CRYPTOCURRENCY, instrument_class=SWAP, quote_currency=USDT, is_inverse=False, price_precision=5, price_increment=0.00010, size_precision=0, size_increment=1, multiplier=1, lot_size=1, margin_init=0.0500, margin_maint=0.0250, maker_fee=0.000200, taker_fee=0.000500, info={'symbol': 'ADAUSDT', 'pair': 'ADAUSDT', 'contractType': 'PERPETUAL', 'deliveryDate': 4133404800000, 'onboardDate': 1569398400000, 'status': 'TRADING', 'maintMarginPercent': '2.5000', 'requiredMarginPercent': '5.0000', 'baseAsset': 'ADA', 'quoteAsset': 'USDT', 'marginAsset': 'USDT', 'pricePrecision': 5, 'quantityPrecision': 0, 'baseAssetPrecision': 8, 'quotePrecision': 8, 'underlyingType': 'COIN', 'underlyingSubType': ['Layer-1'], 'settlePlan': None, 'triggerProtect': '0.0500', 'liquidationFee': '0.010000', 'marketTakeBound': '0.05', 'filters': [{'filterType': 'PRICE_FILTER', 'minPrice': '0.01740', 'maxPrice': '20000', 'tickSize': '0.00010', 'multi

## 3. Write a trading strategy

NautilusTrader includes many indicators built-in, in this example we will use the VWAP indicator to 
build a simple trading strategy.

You can read more about [VWAP here](), this 
indicator merely serves as an example without any expected alpha. There is also a way of
registering indicators to receive certain data types, however in this example we manually pass the received
`QuoteTick` to the indicator in the `on_quote_tick` method.

In [3]:
# -------------------------------------------------------------------------------------------------
#  VWAP Multi-Timeframe Trading Strategy
#  使用VWAP在4小時和15分鐘時間框架上進行交易
# -------------------------------------------------------------------------------------------------

from collections import deque
from decimal import Decimal
from typing import Optional

import numpy as np
from nautilus_trader.common.enums import LogColor
from nautilus_trader.config import StrategyConfig
from nautilus_trader.core.datetime import unix_nanos_to_dt
from nautilus_trader.indicators.vwap import VolumeWeightedAveragePrice
from nautilus_trader.model.data import Bar, BarType
from nautilus_trader.model.enums import OrderSide, TimeInForce
from nautilus_trader.model.events import PositionClosed, PositionOpened
from nautilus_trader.model.identifiers import InstrumentId, Venue
from nautilus_trader.trading.strategy import Strategy


class VWAPStrategyConfig(StrategyConfig, frozen=True):
    """
    Configuration for the VWAP multi-timeframe strategy.
    """

    instrument_id: str
    bar_type_15min: str
    bar_type_4h: str
    vwap_period_15min: int = 100  # Approximately one trading day (for 15min bars)
    vwap_period_4h: int = 30  # Approximately 5 trading days (for 4h bars)
    std_dev_multiplier: float = 2.0  # Standard deviation multiplier for VWAP bands
    entry_volume_threshold: float = 1.5  # Volume threshold compared to average
    risk_per_trade: float = 0.02  # 2% risk per trade
    time_exit_hours: int = (
        24 * 7  # Exit trade after 24 hours * 7 if not stopped out/taken profit
    )


class VWAPMultiTimeframeStrategy(Strategy):
    """
    VWAP multi-timeframe strategy that uses 4-hour and 15-minute bars.

    The strategy:
    1. Uses 4-hour VWAP to determine the overall trend
    2. Uses 15-minute VWAP for entry and exit signals
    3. Uses VWAP standard deviation bands for profit targets and stop losses
    4. Implements volume filters for signal confirmation
    5. Includes risk management with fixed percentage risk per trade
    """

    def __init__(self, config: VWAPStrategyConfig):
        """
        Initialize a new instance of the VWAPMultiTimeframeStrategy.
        """
        super().__init__(config=config)

        # Configuration
        self.bar_type_15min = BarType.from_str(config.bar_type_15min)
        self.bar_type_4h = BarType.from_str(config.bar_type_4h)

        # VWAP indicators
        self.vwap_15min = VolumeWeightedAveragePrice()
        self.vwap_4h = VolumeWeightedAveragePrice()

        # Data storage for calculations
        self.bars_15min = []
        self.bars_4h = []
        self.volumes_15min = deque(maxlen=20)  # For volume average calculation

        # Track last VWAP values for crossover detection
        self.last_15min_price = 0.0
        self.last_15min_vwap = 0.0

        # Standard deviation bands for the 15-min timeframe
        self.upper_band_15min = 0.0
        self.lower_band_15min = 0.0

        # Tracking flags
        self.in_position = False
        self.position_side = None
        self.entry_time = None
        self.current_position_id = None

        # Statistics
        self.trades_total = 0
        self.trades_won = 0
        self.trades_lost = 0

    def on_start(self):
        """
        Actions to perform when the strategy starts.
        """
        self.log.info("VWAP Multi-Timeframe Strategy starting...")
        self.instrument = self.cache.instrument(
            InstrumentId.from_str(self.config.instrument_id)
        )
        # Subscribe to 15-minute bars
        self.subscribe_bars(self.bar_type_15min)

        # Subscribe to 4-hour bars (using bar aggregation if needed)
        try:
            # If 4-hour bars need to be created through aggregation from 15-min bars
            bar_type_4h = f"{self.bar_type_4h}@-15-MINUTE-LAST-INTERNAL"
            self.subscribe_bars(BarType.from_str(bar_type_4h))
            self.log.info(
                "4-hour bars are not available directly, aggregating from 15-minute bars."
            )
        except Exception:
            # If 4-hour bars are available directly
            self.subscribe_bars(self.bar_type_4h)
            self.log.info("4-hour bars are available directly, no aggregation needed.")
        self.log.info(f"Subscribed to 4-hour bars: {self.bar_type_4h}")

        # Register the VWAP indicators to receive bar data
        self.register_indicator_for_bars(self.bar_type_15min, self.vwap_15min)
        self.register_indicator_for_bars(self.bar_type_4h, self.vwap_4h)

    def on_bar(self, bar: Bar) -> None:
        """
        Actions to perform when a new bar is received.

        Parameters
        ----------
        bar : Bar
            The update bar.
        """
        # Process bar based on timeframe
        if bar.bar_type == self.bar_type_15min:
            self._process_15min_bar(bar)
        elif bar.bar_type == self.bar_type_4h:
            self._process_4h_bar(bar)

    def _process_15min_bar(self, bar: Bar) -> None:
        """
        Process a 15-minute bar update.
        """
        # Store the bar and update volume history
        self.bars_15min.append(bar)
        self.volumes_15min.append(float(bar.volume.as_double()))

        # Current price and VWAP values
        current_price = float(bar.close.as_double())
        self.last_15min_price = current_price

        # Wait until both indicators are initialized
        if not self.vwap_15min.initialized or not self.vwap_4h.initialized:
            self.log.info(
                "Waiting for VWAP indicators to initialize...", color=LogColor.BLUE
            )
            return

        # Store current VWAP values
        current_15min_vwap = self.vwap_15min.value
        current_4h_vwap = self.vwap_4h.value

        # Calculate VWAP standard deviation bands for 15-min timeframe
        if len(self.bars_15min) >= self.config.vwap_period_15min:
            # Calculate standard deviation
            recent_bars = self.bars_15min[-self.config.vwap_period_15min :]
            prices = [
                np.divide(
                    (
                        float(b.high.as_double())
                        + float(b.low.as_double())
                        + float(b.close.as_double())
                    ),
                    3.0,
                )
                for b in recent_bars
            ]
            std_dev = np.std(prices)

            # Set bands
            self.upper_band_15min = current_15min_vwap + (
                std_dev * self.config.std_dev_multiplier
            )
            self.lower_band_15min = current_15min_vwap - (
                std_dev * self.config.std_dev_multiplier
            )

            # Log VWAP and bands
            self.log.info(
                f"15min VWAP: {current_15min_vwap:.5f}, "
                f"Upper band: {self.upper_band_15min:.5f}, "
                f"Lower band: {self.lower_band_15min:.5f}",
                color=LogColor.CYAN,
            )

        # Detect 15-min VWAP crossover (if we have previous values)
        if np.not_equal(self.last_15min_vwap, 0.0):
            # Calculate average volume
            avg_volume = (
                np.divide(sum(self.volumes_15min), len(self.volumes_15min))
                if self.volumes_15min
                else 0.0
            )
            current_volume = float(bar.volume.as_double())
            volume_ratio = (
                np.divide(current_volume, avg_volume)
                if np.not_equal(avg_volume, 0.0)
                else 0.0
            )

            # Log volume analysis
            self.log.info(
                f"Volume: {current_volume:.2f}, Avg Volume: {avg_volume:.2f}, "
                f"Ratio: {volume_ratio:.2f}, Threshold: {self.config.entry_volume_threshold:.2f}",
                color=LogColor.YELLOW,
            )

            # See if we need to exit based on time
            if self.in_position and self.entry_time:
                bar_time = unix_nanos_to_dt(bar.ts_event)
                # Check if position has been open for more than time_exit_hours
                elapsed_time = bar_time - self.entry_time
                if elapsed_time.total_seconds() > (self.config.time_exit_hours * 3600):
                    self.log.info(
                        f"Time-based exit triggered after {elapsed_time.total_seconds()/3600:.1f} hours",
                        color=LogColor.MAGENTA,
                    )
                    self._exit_position()

            # Check if we're in a position for exit signals
            if self.in_position:
                # Exit long position
                if self.position_side == OrderSide.BUY:
                    # If price rises above upper band, take profit
                    if current_price >= self.upper_band_15min:
                        self.log.info(
                            f"Take profit triggered: Price {current_price:.5f} >= Upper band {self.upper_band_15min:.5f}",
                            color=LogColor.GREEN,
                        )
                        self._exit_position()
                    # If price falls below VWAP, stop loss
                    elif current_price < current_15min_vwap:
                        self.log.info(
                            f"Stop loss triggered: Price {current_price:.5f} < VWAP {current_15min_vwap:.5f}",
                            color=LogColor.RED,
                        )
                        self._exit_position()

                # Exit short position
                elif self.position_side == OrderSide.SELL:
                    # If price falls below lower band, take profit
                    if current_price <= self.lower_band_15min:
                        self.log.info(
                            f"Take profit triggered: Price {current_price:.5f} <= Lower band {self.lower_band_15min:.5f}",
                            color=LogColor.GREEN,
                        )
                        self._exit_position()
                    # If price rises above VWAP, stop loss
                    elif current_price > current_15min_vwap:
                        self.log.info(
                            f"Stop loss triggered: Price {current_price:.5f} > VWAP {current_15min_vwap:.5f}",
                            color=LogColor.RED,
                        )
                        self._exit_position()

            # Check for entry signals if we're not in a position
            elif not self.in_position:
                # Uptrend in 4-hour timeframe: current price above 4h VWAP
                uptrend_4h = current_price > current_4h_vwap
                # Downtrend in 4-hour timeframe: current price below 4h VWAP
                downtrend_4h = current_price < current_4h_vwap

                # 15-min price crossing above VWAP
                cross_above = (
                    self.last_15min_price > current_15min_vwap
                    and self.last_15min_price <= self.last_15min_vwap
                )
                # 15-min price crossing below VWAP
                cross_below = (
                    self.last_15min_price < current_15min_vwap
                    and self.last_15min_price >= self.last_15min_vwap
                )

                # Volume is above threshold
                volume_check = volume_ratio >= self.config.entry_volume_threshold

                # Long signal: 4h uptrend + 15min cross above VWAP + high volume
                if uptrend_4h and cross_above and volume_check:
                    self.log.info(
                        "LONG SIGNAL: 4h uptrend + 15min cross above VWAP + high volume",
                        color=LogColor.GREEN,
                    )
                    self._enter_position(OrderSide.BUY, bar)

                # Short signal: 4h downtrend + 15min cross below VWAP + high volume
                elif downtrend_4h and cross_below and volume_check:
                    self.log.info(
                        "SHORT SIGNAL: 4h downtrend + 15min cross below VWAP + high volume",
                        color=LogColor.RED,
                    )
                    self._enter_position(OrderSide.SELL, bar)

        # Update last VWAP value for next comparison
        self.last_15min_vwap = current_15min_vwap

    def _process_4h_bar(self, bar: Bar) -> None:
        """
        Process a 4-hour bar update.
        """
        # Store the bar
        self.bars_4h.append(bar)

        # Log 4-hour VWAP if available
        if self.vwap_4h.initialized:
            self.log.info(
                f"4h VWAP updated: {self.vwap_4h.value:.5f} at {unix_nanos_to_dt(bar.ts_event)}",
                color=LogColor.MAGENTA,
            )

    def _enter_position(self, side: OrderSide, bar: Bar) -> None:
        """
        Enter a new position.

        Parameters
        ----------
        side : OrderSide
            The order side (BUY or SELL).
        bar : Bar
            The current bar.
        """
        if self.in_position:
            self.log.warning("Already in position, cannot enter new position.")
            return

        # Calculate position size based on risk percentage
        account_balance = self.get_account_balance(self.instrument.quote_currency)
        if account_balance is None:
            self.log.error("Unable to determine account balance.")
            return

        current_price = float(bar.close.as_double())

        # Calculate stop loss price
        if side == OrderSide.BUY:
            stop_price = self.lower_band_15min
        else:  # SELL
            stop_price = self.upper_band_15min

        # Calculate risk per trade in currency
        risk_amount = float(account_balance) * self.config.risk_per_trade

        # Calculate position size based on risk
        price_distance = abs(current_price - float(stop_price))
        if price_distance <= 0:
            self.log.error(f"Invalid price distance: {price_distance}. Aborting trade.")
            return

        position_size = risk_amount / price_distance
        position_qty = self.instrument.make_qty(Decimal(str(position_size)))

        # Adjust position size if it's below the minimum lot size
        min_qty = self.instrument.min_quantity
        if position_qty < min_qty:
            position_qty = min_qty
            self.log.warning(
                f"Position size adjusted to minimum quantity: {position_qty}"
            )

        # Create market order for entry
        order = self.order_factory.market(
            instrument_id=self.instrument.id,
            order_side=side,
            # quantity=self.instrument.calculate_base_quantity(position_qty, bar.close),
            quantity=position_qty,
            time_in_force=TimeInForce.GTC,  # Immediate or Cancel
            reduce_only=False,
        )

        # Submit the order
        self.submit_order(order)
        self.log.info(
            f"Submitted {side} order: {order}",
            color=LogColor.GREEN if side == OrderSide.BUY else LogColor.RED,
        )

        # Update tracking variables
        self.in_position = True
        self.position_side = side
        self.entry_time = unix_nanos_to_dt(bar.ts_event)
        self.trades_total += 1

    def _exit_position(self) -> None:
        """
        Exit the current position.
        """
        if not self.in_position or self.position_side is None:
            self.log.warning("No position to exit.")
            return

        # Create opposing market order to close the position
        exit_side = (
            OrderSide.SELL if self.position_side == OrderSide.BUY else OrderSide.BUY
        )

        # Get the current position size
        position = self.portfolio.net_position(self.instrument.id)
        if position == Decimal("0"):
            self.log.warning("No position to exit.")
            # Reset tracking variables anyway
            self.in_position = False
            self.position_side = None
            self.entry_time = None
            self.current_position_id = None
            return
        if exit_side == OrderSide.BUY:
            position = -position
        # Create market order for exit
        order = self.order_factory.market(
            instrument_id=self.instrument.id,
            order_side=exit_side,
            quantity=self.instrument.make_qty(position),
            time_in_force=TimeInForce.GTC,  # Immediate or Cancel
            reduce_only=False,  # Ensure we only reduce position, not open new one
        )

        # Submit the order
        self.submit_order(order)
        self.log.info(
            f"Submitted exit {exit_side} order: {order}", color=LogColor.YELLOW
        )

        # We'll reset tracking variables when we receive the position closed event

    def on_position_opened(self, event: PositionOpened) -> None:
        """
        Callback for position opened event.

        Parameters
        ----------
        event : PositionOpened
            The position opened event.
        """
        if event.instrument_id != self.instrument.id:
            return  # Not our instrument

        self.log.info(f"Position opened: {event}")
        self.current_position_id = event.position_id

    def on_position_closed(self, event: PositionClosed) -> None:
        """
        Callback for position closed event.

        Parameters
        ----------
        event : PositionClosed
            The position closed event.
        """
        if event.instrument_id != self.instrument.id:
            return  # Not our instrument

        self.log.info(f"Position closed: {event}")

        # Check if this is our current position
        if self.current_position_id == event.position_id:
            # Update trade statistics
            if float(event.realized_pnl) >= 0:
                self.trades_won += 1
                self.log.info(
                    f"Trade won: Realized P&L = {event.realized_pnl}",
                    color=LogColor.GREEN,
                )
            else:
                self.trades_lost += 1
                self.log.info(
                    f"Trade lost: Realized P&L = {event.realized_pnl}",
                    color=LogColor.RED,
                )

            # Reset tracking variables
            self.in_position = False
            self.position_side = None
            self.entry_time = None
            self.current_position_id = None

            # Log trade statistics
            win_rate = (
                (self.trades_won / self.trades_total) * 100
                if self.trades_total > 0
                else 0
            )
            self.log.info(
                f"Trade statistics: Won={self.trades_won}, Lost={self.trades_lost}, "
                f"Total={self.trades_total}, Win rate={win_rate:.2f}%",
                color=LogColor.BLUE,
            )

    def get_account_balance(self, currency) -> Optional[Decimal]:
        """
        Get the account balance for the specified currency.

        Parameters
        ----------
        currency : Currency
            The currency to check.

        Returns
        -------
        Optional[Decimal]
            The account balance if available, None otherwise.
        """
        try:
            return self.portfolio.account(Venue("BINANCE")).balance_total()
        except Exception as e:
            self.log.error(f"Error getting account balance: {e}")
            return None

    def on_stop(self) -> None:
        """
        Actions to perform when the strategy stops.
        """
        self.log.info("VWAP Multi-Timeframe Strategy stopped.")

        # Log final statistics
        self.log.info(f"Total trades: {self.trades_total}")
        self.log.info(f"Won trades: {self.trades_won}")
        self.log.info(f"Lost trades: {self.trades_lost}")

        win_rate = (
            (self.trades_won / self.trades_total) * 100 if self.trades_total > 0 else 0
        )
        self.log.info(f"Win rate: {win_rate:.2f}%")

## Configuring backtests

Now that we have a trading strategy and data, we can begin to configure a backtest run. Nautilus uses a `BacktestNode` 
to orchestrate backtest runs, which requires some setup. This may seem a little complex at first, 
however this is necessary for the capabilities that Nautilus strives for.

To configure a `BacktestNode`, we first need to create an instance of a `BacktestRunConfig`, configuring the 
following (minimal) aspects of the backtest:

- `engine`: The engine for the backtest representing our core system, which will also contain our strategies
- `venues`: The simulated venues (exchanges or brokers) available in the backtest
- `data`: The input data we would like to perform the backtest on

There are many more configurable features which will be described later in the docs, for now this will get us up and running.

## 4. Configure venue

First, we create a venue configuration. For this example we will create a simulated FX ECN. 
A venue needs a name which acts as an ID (in this case `SIM`), as well as some basic configuration, e.g. 
the account type (`CASH` vs `MARGIN`), an optional base currency, and starting balance(s).

:::note
FX trading is typically done on margin with Non-Deliverable Forward, Swap or CFD type instruments.
:::

In [4]:
# Define the instrument for the strategy
venue = BacktestVenueConfig(
    name="BINANCE",
    oms_type="NETTING",
    account_type="MARGIN",
    starting_balances=["1000 USDT"],
    base_currency="USDT",
    default_leverage=Decimal("10.0"),
)

## 5. Configure data

We need to know about the instruments that we would like to load data for, we can use the `ParquetDataCatalog` for this.

In [5]:
instruments = catalog.instruments()
instruments

[CryptoPerpetual(id=ADAUSDT-PERP.BINANCE, raw_symbol=ADAUSDT, asset_class=CRYPTOCURRENCY, instrument_class=SWAP, quote_currency=USDT, is_inverse=False, price_precision=5, price_increment=0.00010, size_precision=0, size_increment=1, multiplier=1, lot_size=1, margin_init=0.0500, margin_maint=0.0250, maker_fee=0.000200, taker_fee=0.000500, info={'symbol': 'ADAUSDT', 'pair': 'ADAUSDT', 'contractType': 'PERPETUAL', 'deliveryDate': 4133404800000, 'onboardDate': 1569398400000, 'status': 'TRADING', 'maintMarginPercent': '2.5000', 'requiredMarginPercent': '5.0000', 'baseAsset': 'ADA', 'quoteAsset': 'USDT', 'marginAsset': 'USDT', 'pricePrecision': 5, 'quantityPrecision': 0, 'baseAssetPrecision': 8, 'quotePrecision': 8, 'underlyingType': 'COIN', 'underlyingSubType': ['Layer-1'], 'settlePlan': None, 'triggerProtect': '0.0500', 'liquidationFee': '0.010000', 'marketTakeBound': '0.05', 'filters': [{'filterType': 'PRICE_FILTER', 'minPrice': '0.01740', 'maxPrice': '20000', 'tickSize': '0.00010', 'multi

Next, we need to configure the data for the backtest. Nautilus is built to be very flexible when it 
comes to loading data for backtests, however this also means some configuration is required.

For each tick type (and instrument), we add a `BacktestDataConfig`. In this instance we are simply 
adding the `QuoteTick`(s) for our EUR/USD instrument:

In [6]:
start = "2024-01-01"
end = "2024-12-31"
data_15min = BacktestDataConfig(
    catalog_path=str(catalog.path),
    data_cls=Bar,
    instrument_id=instruments[3].id,
    start_time=start,
    end_time=end,
    bar_spec="15-MINUTE-LAST",
)
data_1min = BacktestDataConfig(
    catalog_path=str(catalog.path),
    data_cls=Bar,
    instrument_id=instruments[3].id,
    start_time=start,
    end_time=end,
    bar_spec="1-MINUTE-LAST",
)

## 6. Configure engine

Then, we need a `BacktestEngineConfig` which represents the configuration of our core trading system.
Here we need to pass our trading strategies, we can also adjust the log level 
and configure many other components (however, it's also fine to use the defaults):

Strategies are added via the `ImportableStrategyConfig`, which enables importing strategies from arbitrary files or 
user packages. In this instance, our `MACDStrategy` is defined in the current module, which python refers to as `__main__`.

In [7]:
# NautilusTrader currently exceeds the rate limit for Jupyter notebook logging (stdout output),
# this is why the `log_level` is set to "ERROR". If you lower this level to see
# more logging then the notebook will hang during cell execution. A fix is currently
# being investigated which involves either raising the configured rate limits for
# Jupyter, or throttling the log flushing from Nautilus.
# https://github.com/jupyterlab/jupyterlab/issues/12845
# https://github.com/deshaw/jupyterlab-limit-output
engine = BacktestEngineConfig(
    strategies=[
        ImportableStrategyConfig(
            strategy_path="__main__:VWAPMultiTimeframeStrategy",
            config_path="__main__:VWAPStrategyConfig",
            config={
                "instrument_id": str(instruments[3].id),
                "bar_type_15min": f"{instruments[3].id}-15-MINUTE-LAST-EXTERNAL",
                "bar_type_4h": f"{instruments[3].id}-4-HOUR-LAST-EXTERNAL",
                "vwap_period_15min": 100,  # Approximately one trading day (for 15min bars)
                "vwap_period_4h": 30,  # Approximately 5 trading days (for 4h bars)
                "std_dev_multiplier": 2.0,  # Standard deviation multiplier for VWAP bands
                "entry_volume_threshold": 1.5,  # Volume threshold compared to average
                "risk_per_trade": 0.1,  # 10% risk per trade
                "time_exit_hours": (
                    24 * 7  # Exit trade after 24 hours if not stopped out/taken profit
                ),
            },
        )
    ],
    logging=LoggingConfig(log_level="ERROR"),
)

## 7. Run backtest

We can now pass our various config pieces to the `BacktestRunConfig`. This object now contains the 
full configuration for our backtest.

In [8]:
config = BacktestRunConfig(
    engine=engine,
    venues=[venue],
    data=[data_15min, data_1min],
)

The `BacktestNode` class will orchestrate the backtest run. The reason for this separation between 
configuration and execution is the `BacktestNode`, which enables running multiple configurations (different 
parameters or batches of data). We are now ready to run some backtests.

In [9]:
from nautilus_trader.backtest.results import BacktestResult


node = BacktestNode(configs=[config])

# Runs one or many configs synchronously
results: list[BacktestResult] = node.run()

## 8. Analyze results

Now that the run is complete, we can also directly query for the `BacktestEngine`(s) used internally by the `BacktestNode`
by using the run configs ID. 

The engine(s) can provide additional reports and information.

In [10]:
from nautilus_trader.backtest.engine import BacktestEngine

engine: BacktestEngine = node.get_engine(config.id)
order_fills_report = pl.DataFrame(engine.trader.generate_order_fills_report())
order_fills_report.write_json("order_fills_report.json")
print(order_fills_report)

shape: (122, 31)
┌────────────┬────────────┬────────────┬────────────┬───┬──────┬───────────┬───────────┬───────────┐
│ trader_id  ┆ strategy_i ┆ instrument ┆ venue_orde ┆ … ┆ tags ┆ init_id   ┆ ts_init   ┆ ts_last   │
│ ---        ┆ d          ┆ _id        ┆ r_id       ┆   ┆ ---  ┆ ---       ┆ ---       ┆ ---       │
│ str        ┆ ---        ┆ ---        ┆ ---        ┆   ┆ str  ┆ str       ┆ datetime[ ┆ datetime[ │
│            ┆ str        ┆ str        ┆ str        ┆   ┆      ┆           ┆ ns, UTC]  ┆ ns, UTC]  │
╞════════════╪════════════╪════════════╪════════════╪═══╪══════╪═══════════╪═══════════╪═══════════╡
│ BACKTESTER ┆ VWAPMultiT ┆ LTCUSDT-PE ┆ BINANCE-1- ┆ … ┆ null ┆ ed49115f- ┆ 2024-01-0 ┆ 2024-01-0 │
│ -001       ┆ imeframeSt ┆ RP.BINANCE ┆ 001        ┆   ┆      ┆ 6383-4729 ┆ 4         ┆ 4         │
│            ┆ rategy-000 ┆            ┆            ┆   ┆      ┆ -8d8c-05a ┆ 00:00:00  ┆ 00:00:00  │
│            ┆            ┆            ┆            ┆   ┆      ┆ cc9…     

In [11]:
positions_report = pl.DataFrame(engine.trader.generate_positions_report())
positions_report.write_json("positions_report.json")
print(positions_report)

shape: (61, 20)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ trader_id ┆ strategy_ ┆ instrumen ┆ account_i ┆ … ┆ avg_px_cl ┆ commissio ┆ realized_ ┆ realized │
│ ---       ┆ id        ┆ t_id      ┆ d         ┆   ┆ ose       ┆ ns        ┆ return    ┆ _pnl     │
│ str       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│           ┆ str       ┆ str       ┆ str       ┆   ┆ f64       ┆ list[str] ┆ f64       ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ BACKTESTE ┆ VWAPMulti ┆ LTCUSDT-P ┆ BINANCE-0 ┆ … ┆ 65.67     ┆ ["2.38582 ┆ 0.00721   ┆ 14.75084 │
│ R-001     ┆ Timeframe ┆ ERP.BINAN ┆ 01        ┆   ┆           ┆ 554       ┆           ┆ 446 USDT │
│           ┆ Strategy- ┆ CE        ┆           ┆   ┆           ┆ USDT"]    ┆           ┆          │
│           ┆ 000       ┆           ┆           ┆   ┆           ┆          

In [12]:
account_report = pl.DataFrame(engine.trader.generate_account_report(Venue("BINANCE")))
account_report.write_json("account_report.json")
print(account_report)

shape: (157, 10)
┌────────────┬───────────┬───────────┬──────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ total      ┆ locked    ┆ free      ┆ currency ┆ … ┆ base_curr ┆ margins   ┆ reported ┆ info      │
│ ---        ┆ ---       ┆ ---       ┆ ---      ┆   ┆ ency      ┆ ---       ┆ ---      ┆ ---       │
│ str        ┆ str       ┆ str       ┆ str      ┆   ┆ ---       ┆ list[stru ┆ bool     ┆ struct[0] │
│            ┆           ┆           ┆          ┆   ┆ str       ┆ ct[5]]    ┆          ┆           │
╞════════════╪═══════════╪═══════════╪══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ 1000.00000 ┆ 0E-8      ┆ 1000.0000 ┆ USDT     ┆ … ┆ USDT      ┆ []        ┆ true     ┆ {}        │
│ 000        ┆           ┆ 0000      ┆          ┆   ┆           ┆           ┆          ┆           │
│ 998.811371 ┆ 0E-8      ┆ 998.81137 ┆ USDT     ┆ … ┆ USDT      ┆ []        ┆ false    ┆ {}        │
│ 40         ┆           ┆ 140       ┆          ┆   ┆           ┆         

In [13]:
node.dispose()